<a href="https://colab.research.google.com/github/hursoo/big_k-modern_1/blob/main/gb_011_scraping(big1_251)_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.필요한 도구와 라이브러리 설치

In [ ]:
# 구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 경로 지정

file_path = '/content/drive/MyDrive/_big1_25-1_code/'

In [ ]:
import ssl
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 2.기사정보 다운로드

사전 작업
- 국편 한국사데이터베이스/개벽에서 기사정보 다운로드 = "근현대잡지자료_20250315172708.txt"

In [ ]:
gisa_info = pd.read_csv(file_path + 'data/근현대잡지자료_20250315172708.txt', sep = '^', encoding='utf-8')
gisa_info

# 3.기사정보로부터 주요논설 선별
- 오프라인(코드 바깥) 작업
- 2000여개 기사를 보고 필요한 '주요논설'을 선별

# 4.주요논설 334개 목록파일 불러옴

In [ ]:
# 주요 논설 id
r334_info = pd.read_excel(file_path + 'data/gb_data_2.1.xlsx', sheet_name = 'ron_info')
r334_info

In [ ]:
urls = r334_info['url'].tolist()
urls[:3]

# 5.주요 논설 원문 스크래핑
- 기사정보와 주요논설 목록을 활용.
- url 주소를 이용하여 웹 페이지에 접속하고, BeautifulSoup을 사용하여 원하는 부분의 데이터를 추출

In [ ]:
def get_contents(urls, n):
    results = []

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    for url in urls[:n]:
        webpage = urlopen(url, context=ctx)
        r_id = url[-16:]

        bsobj = BeautifulSoup(webpage.read(), 'html.parser')
        List1 = bsobj.findAll('div', {'id': 'cont_view'})

        for z in List1:
            z1 = z.get_text('\n', strip=True)
            results.append([r_id, z1])

    result_df = pd.DataFrame(results, columns=['r_id', 'content'])
    return result_df

In [ ]:
contents_df = get_contents(urls, 10)
contents_df

In [ ]:
# 본문 내용 확인 (첫 행)
#contents_5_df.loc[3,'Content']
#print(contents_df.loc[3,'content'])

# 6.데이터 저장

In [ ]:
r334_info1 = r334_info.drop('r_id', axis=1) # 두 df의 중복열 중 미리 한쪽을 제거
combi_df = pd.merge(r334_info1, contents_df, left_on='r_id_raw', right_on = 'r_id', how='inner')
combi_df1 = combi_df[['r_id', 'r_id_raw', 'title', 'writer', 'gisa_class', 'date', 'url', 'year', 'content']]
combi_df1

In [ ]:
# 저장
combi_df1.to_excel(file_path + 'result/ron10_data.xlsx', index=False)  # 인덱스를 저장하지 않도록 설정

# The End of Note